In this assignment, we will implement Bernoulli Naive Bayes and Multinomial Naive Bayes, and apply them for text classification. \\
We will experiment on the 20 newsgroups text dataset. It comprises around 18000 newsgroups posts on 20 topics split in two subsets: one for training (or development) and the other one for testing (or for performance evaluation). \\
First, we load the dataset from sklearn

In [44]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

You can access to text by `data` property. For labels, their name and corresponding numeric values are stored in `target_names` and `target`. \\
Let's take a look at our data

In [45]:
len(newsgroups_train.data)

11314

In [46]:
newsgroups_train.target, newsgroups_train.target_names

(array([7, 4, 4, ..., 3, 1, 8]),
 ['alt.atheism',
  'comp.graphics',
  'comp.os.ms-windows.misc',
  'comp.sys.ibm.pc.hardware',
  'comp.sys.mac.hardware',
  'comp.windows.x',
  'misc.forsale',
  'rec.autos',
  'rec.motorcycles',
  'rec.sport.baseball',
  'rec.sport.hockey',
  'sci.crypt',
  'sci.electronics',
  'sci.med',
  'sci.space',
  'soc.religion.christian',
  'talk.politics.guns',
  'talk.politics.mideast',
  'talk.politics.misc',
  'talk.religion.misc'])

In [47]:
newsgroups_train.data[0]

"From: lerxst@wam.umd.edu (where's my thing)\nSubject: WHAT car is this!?\nNntp-Posting-Host: rac3.wam.umd.edu\nOrganization: University of Maryland, College Park\nLines: 15\n\n I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.\n\nThanks,\n- IL\n   ---- brought to you by your neighborhood Lerxst ----\n\n\n\n\n"

When applying machine learning to solve problems, designing algorithm is not the only way to optimize. We can also intervene on data, i.e. data preprocessing, feature selection, etc. In some case, this approach is even better than model optimizing. \\
For this dataset, you can notice that the text have lots of redundant information, for example punctuation, title, etc. We can remove those from our data to get better performance. Here I define a function to remove all punctuation from text.

In [48]:
def remove_tokens(token_list, text):
    for token in token_list:
        text = text.replace(token, '')
    return text

In [49]:
from string import punctuation
preprocessed_text = [remove_tokens(punctuation, text) for text in newsgroups_train.data]

**Assignment 1** : First we have to transform text into numeric feature. You have to build a matrix that counts word occurences in each documents **(0.5pt)**. For fast computing, we only select `30000` words with highest frequency. \\
*Hint* You should use `sklearn.feature_extraction.text.CountVectorizer` and `max_features` argument.

In [50]:
from sklearn.feature_extraction.text import CountVectorizer
num_word = 30000
vectorizer = CountVectorizer(max_features = num_word)
train_data = vectorizer.fit_transform(preprocessed_text)
train_data.shape

(11314, 30000)

Recall that for Naive Bayes, we find label that satisfy 
$$c=\arg\max_{c'} p(c')\prod p(x_i|c')$$
**Assigment 2** : We will derive prior probabilities $p(c')$ from data by computing frequency of class. You have to compute the number of documents in each class in `class_freq` variable, and divide to the total number of documents to get prior probability in `prior_prob` variable **(1pt)**

In [51]:
import numpy as np
classes,class_freq = np.unique(newsgroups_train.target,return_counts = True)
prior_prob = class_freq / np.sum(class_freq)

**Assigment 3** : In this step, we will implement Bernoulli Naive Bayes. Therefore, the conditional probability is probability of that a document with label $c$ has the word $x_i$. \\
To do that, we need the number of documents which has word $x_i$ and label $c$ for every pair $(x_i,c)$. Your task is computing these values and store them in `word_label_freq` variable. It should be a numpy array for fast computing in the next step. **(0.5.pt)** \\
*Hint:* Our `'train_data` features are the number of occurences of words in documents. You can convert them to binary feature that whether a word appears in a document.

In [52]:
occur_matx = (train_data > 0).astype('float')
word_label_freq = np.zeros((20, num_word))
for l in range(20):
    ind = newsgroups_train.target == l
    word_label_freq[l] = occur_matx[ind].sum(axis=0)
word_label_freq 

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 5.,  0.,  0., ..., 10.,  2.,  1.],
       [ 2.,  0.,  1., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  4.,  1., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

**Assigment 4** : 
The conditional probability is computed by dividing the number of documents which has word $x_i$ and label $c$ to the number of documents with label $c$. However, if there is no document which has word $x_i$ and label $c$ in training data, the probability will be zero, which is undesirable. \\
To handle this problem, we can apply Laplace smoothing, then conditional probability will be computed as following
$$p(x_i=1|c) = \frac{N_{ic} + \alpha}{N_c +|V|\alpha}$$
Your task here is implementing this formula with default `alpha=0.1` and then fill in all the probability values in `cond_prob` variable. It should be a numpy array for fast computing in the next step. **(1pt)**

In [53]:
alpha = 0.01

In [54]:
cond_prob =  cond_prob = (word_label_freq + 0.01) / (np.expand_dims(class_freq, 1) + num_word * 0.01)
cond_prob.shape

(20, 30000)

**Assigment 5** : For test data, the conditional probabily follows Bernoulli distribution and is computed by
$$p(x_i|c) = p(x_i=1|c)x_i + p(x_i=0|c)(1 - x_i)$$
Then we multiply with prior probability and select the class with highest value as predicted label. For numerical stability, you should use log probability to compute. **(2pt)** \\
$$c = \arg\max_{c'}(\log p(c')+\sum_{x_i\in text}\log p(x_i|c'))$$
*Hint* Remember to convert test data feature to binary feature as training data.


In [55]:
def find_label(data):
    log_prob = np.log(prior_prob)
    data = (data > 0).toarray()
    in_prob = np.log(cond_prob)
    out_prob = np.log(1 - cond_prob)
    log_prob += (in_prob * data + out_prob * (1 - data)).sum(axis=1) 
    return log_prob.argmax() 

Now we can obtain labels and accuracy score of model on test data

In [56]:
preprocessed_test_text = [remove_tokens(punctuation, text) for text in newsgroups_test.data]
test_data = vectorizer.transform(preprocessed_test_text)

In [57]:
pred = []
from tqdm import tqdm
for text in tqdm(test_data):
    pred.append(find_label(text))

7532it [05:59, 20.96it/s]


In [58]:
from sklearn import metrics
metrics.accuracy_score(pred, newsgroups_test.target)

0.7825278810408922

**Assigment 6** : Next, we will implement Multinomial Naive Bayes. For this model, the conditional probability follows multinomial distribution. We have to estimate conditional probabilities from data as the frequency of words in all documents with a given class. \\
First, you have to count the number of occurences of a word $x_i$ in all document with class $c$ **(0.5pt)**.

In [59]:
word_label_freq = np.zeros((20, num_word))
for l in range(20):
    ind = newsgroups_train.target == l
    word_label_freq[l] = train_data[ind].sum(axis=0)

word_label_freq.shape

(20, 30000)

**Assigment 7** : Your task here is to count the total number of words in all documents of class $c$ to compute probability in next step. **(0.5)pt** \\
*Hint* You can sum the number of occurences of all words in class $c$ that we obtained in last step.

In [60]:
num_word_in_classes = word_label_freq.sum(axis=1)

**Assigment 8** : Now we can compute conditional probability for every pair $(x_i, c)$. Similar to Bernoulli Naive Bayes, we also add Laplace smoothing to avoid zero probability
$$p(x_i|c) = \frac{N_{ic} + \alpha}{N_c +|V|\alpha}$$
where $N_{ic}$ is the number of occurences of word $x_i$ in all documents with class $c$, $N_c$ is the total number of words in all documents of class $c$. \\
For numerical stability, you should compute log of probablity **(1pt)**.

In [61]:
log_cond_prob = np.log(word_label_freq + 0.01) - np.log(np.expand_dims(num_word_in_classes, 1) + num_word * 0.01)

**Assignment 9** : After getting all necessary probabilities, we can find label of new test data. For this task, you have to implement `find_label` function that compute product of prior and conditional probablities, and select the label with highest value. Finally, you can get prediction for all test data and get accuracy score **(3pt)**.

In [62]:
log_prior_prob = np.log(prior_prob)
def find_label(data):
    log_prob = log_prior_prob.copy()
    data = data.toarray()
    log_prob += (log_cond_prob.copy() * data).sum(axis=1)
    return log_prob.argmax()

In [63]:
from sklearn import metrics
pred = []
for text in tqdm(test_data):
    pred.append(find_label(text))
metrics.accuracy_score(pred, newsgroups_test.target)

7532it [02:38, 47.60it/s]


0.8266064790228359

**(Optional)** Try to improve performance of Naive Bayes model in this dataset. You can try everything to do this, i.e. change hyperparameters, preprocess data, ...